<a href="https://colab.research.google.com/github/Hillascher5/nlp-tweets-sentiment-analysis/blob/main/Text_Classification_Tweets_Sentiment_no_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **NLP - Text Classification Project**
Group H - August 2025

Classification of tweets from Twitter that have been manually tagged for sentiment analysis.

In [101]:
# # Needed for Google Colab
# !pip install --quiet evaluate transformers optuna datasets nltk scikit-learn
# !pip install numpy==1.26.4

In [102]:
%env CUDA_LAUNCH_BLOCKING=1

from wordcloud import WordCloud, STOPWORDS
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import Dataset
from optuna.pruners import MedianPruner
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import string
import time
import glob
import nltk
import evaluate
import transformers
import optuna
import wandb
wandb.login()

os.environ["WANDB_PROJECT"] = "tweet-sentiment-classification"
os.environ["WANDB_INIT_TIMEOUT"] = "180"

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

env: CUDA_LAUNCH_BLOCKING=1


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [103]:
# Load data
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_train.csv', encoding='latin1')
df_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nlp_project/Data/Corona_NLP_test.csv', encoding='latin1')

### Pre-processing the Data

The tweets were cleaned by lowercasing (to reduce redundancy), removing stopwords, punctuation, numbers, short words, and applying lemmatization to reduce words to their base form (e.g. running → run). <br>This helps reduce noise and improve model performance.


In [104]:
# # Try agressive pre-processing
# lemmatizer = WordNetLemmatizer()

# def preprocess_text(text):
#     text = str(text).lower()
#     text = re.sub(r"http\S+|www.\S+", "", text)              # Remove URLs
#     text = re.sub(r"[^a-z\s]", "", text)                     # Remove punctuation & numbers
#     tokens = text.split()
#     tokens = [
#         lemmatizer.lemmatize(word)
#         for word in tokens
#         if word not in stop_words
#         and word not in domain_stopwords
#         and len(word) > 2
#     ]
#     return " ".join(tokens)

# df_train["clean_text"] = df_train["OriginalTweet"].apply(preprocess_text)
# df_val["clean_text"] = df_val["OriginalTweet"].apply(preprocess_text)

In [105]:
# Try lighter preprocessing
def light_preprocess(text):
    text = re.sub(r"http\S+|www.\S+", "", text)   # Remove URLs
    text = re.sub(r"@\w+", "", text)              # Remove @mentions
    text = re.sub(r"#(\w+)", r"\1", text)         # Unwrap hashtags
    return text.strip()

df_train["clean_text"] = df_train["OriginalTweet"].apply(light_preprocess)
df_val["clean_text"] = df_val["OriginalTweet"].apply(light_preprocess)

In [106]:
# # Try without pre-processing
# df_train["clean_text"] = df_train["OriginalTweet"]
# df_val["clean_text"] = df_val["OriginalTweet"]

## Fine-Tuning Pretrained Language Models

Apply NLP techniques using transfer learning on our tweets dataset. Specifically, fine-tuning two pretrained transformer-based models from the Hugging Face library — BERT and RoBERTa — on our sentiment classification task. These models will be trained using both standard PyTorch and the Hugging Face API. Model performance will be monitored and tuned using hyperparameter optimization (Optuna) and experiment tracking (Weights & Biases).

**Load Pretrained Models**

Initialize tokenizers and models for BERT and RoBERTa, both widely used transformer architectures for text classification. The classification head is configured based on the number of sentiment labels.

In [107]:
# Choose pretrained models
bert_model_name = "bert-base-uncased"
roberta_model_name = "roberta-base"

sentiment_labels = df_train['Sentiment'].unique()
n_labels = len(sentiment_labels)

# Load BERT tokenizer and model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name, num_labels=n_labels)

# Load RoBERTa tokenizer and model
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name, num_labels=n_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Encode Sentiment Labels**

Map each unique sentiment label to a numeric ID for model compatibility, and apply this mapping to both training and validation datasets.

In [108]:
# Mapping sentiments to unique numeric IDs
unique_labels = sorted(df_train["Sentiment"].unique())

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()}

df_train["label"] = df_train["Sentiment"].map(label2id)
df_val["label"] = df_val["Sentiment"].map(label2id)

In [109]:
# Create stratified subset for Optuna
train_subset_df, _ = train_test_split(
    df_train[["clean_text", "label"]],
    train_size=2000,
    stratify=df_train["label"],
    random_state=42
    )
val_subset_df, _ = train_test_split(
    df_val[["clean_text", "label"]],
    train_size=500,
    stratify=df_val["label"],
    random_state=42
    )

**Tokenization**

Define tokenization functions for BERT and RoBERTa to preprocess text with truncation and fixed padding.

Transform training and validation DataFrames into Dataset objects compatible with Hugging Face workflows.

Apply tokenization to training and validation datasets using each model's tokenizer.

In [110]:
# Tokenize function
def tokenize_function_bert(examples):
    return bert_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

def tokenize_function_roberta(examples):
    return roberta_tokenizer(examples["clean_text"], truncation=True, padding='max_length', max_length=64)

In [111]:
# Convert DataFrame to Hugging Face Dataset
hf_dataset_train = Dataset.from_pandas(df_train[["clean_text", "label"]])
hf_dataset_val = Dataset.from_pandas(df_val[["clean_text", "label"]])

hf_subset_train = Dataset.from_pandas(train_subset_df)
hf_subset_val = Dataset.from_pandas(val_subset_df)

In [112]:
# Tokenize subsets
# Tokenize for BERT
tokenized_bert_train_sub = hf_subset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val_sub = hf_subset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train_sub = hf_subset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val_sub = hf_subset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val_sub.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [113]:
# Tokenize full dataset
# Tokenize for BERT
tokenized_bert_train = hf_dataset_train.map(tokenize_function_bert, batched=True)
tokenized_bert_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_bert_val = hf_dataset_val.map(tokenize_function_bert, batched=True)
tokenized_bert_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Tokenize for RoBERTa
tokenized_roberta_train = hf_dataset_train.map(tokenize_function_roberta, batched=True)
tokenized_roberta_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_roberta_val = hf_dataset_val.map(tokenize_function_roberta, batched=True)
tokenized_roberta_val.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/41157 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

Map:   0%|          | 0/41157 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [114]:
# # Define evaluation metric, Set up an evaluation function using accuracy as the metric to assess model performance during training.

# accuracy_metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=1)
#     return accuracy_metric.compute(predictions=predictions, references=labels)

In [115]:
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"]
    }

**Use Small Subsets for Quick Evaluation**

Select shuffled samples from each training and validation dataset for both BERT and RoBERTa. This allows faster experimentation during model development.

In [116]:
USE_SMALL_DATASET = True
# if USE_SMALL_DATASET:
#   n_samples_train = 2000
#   n_samples_val = 500

#   train_dataset_bert = tokenized_bert_train.shuffle(seed=42).select(range(n_samples_train))
#   val_dataset_bert = tokenized_bert_val.shuffle(seed=42).select(range(n_samples_val))

#   train_dataset_roberta = tokenized_roberta_train.shuffle(seed=42).select(range(n_samples_train))
#   val_dataset_roberta = tokenized_roberta_val.shuffle(seed=42).select(range(n_samples_val))

# else:
#   n_samples_train = len(tokenized_bert_train)
#   n_samples_val = len(tokenized_bert_val)

#   train_dataset_bert = tokenized_bert_train
#   val_dataset_bert = tokenized_bert_val

#   train_dataset_roberta = tokenized_roberta_train
#   val_dataset_roberta = tokenized_roberta_val

**Define Training Arguments**

Configure hyperparameters and training settings for both BERT and RoBERTa models, including batch size, number of epochs, learning rate, and evaluation strategy.

In [117]:
# # Define hyperparameters
# run_time = f"run-{int(time.time())}"

# num_epochs_bert = 5
# learning_rate_bert = 4.037872385196561e-05
# batch_size_bert = 16

# bert_training_args = TrainingArguments(
#     output_dir="./results/bert/{run_time}",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=batch_size_bert,
#     per_device_eval_batch_size=batch_size_bert,
#     num_train_epochs=num_epochs_bert,
#     learning_rate=learning_rate_bert,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     logging_dir="./logs/bert/{run_time}",
#     run_name = f"bert-ep{num_epochs_bert}-lr{learning_rate_bert}-bs{batch_size_bert}-samples{n_samples_train}-run{int(time.time())}-no_preprocess",
#     report_to="wandb"
# )

In [118]:
# # Define hyperparameters
# num_epochs_roberta = 4
# learning_rate_roberta = 4e-5
# batch_size_roberta = 16

# roberta_training_args = TrainingArguments(
#     output_dir="./results/roberta/{run_time}",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=batch_size_roberta,
#     per_device_eval_batch_size=batch_size_roberta,
#     num_train_epochs=num_epochs_roberta,
#     learning_rate=learning_rate_roberta,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     logging_dir="./logs/roberta/{run_time}",
#     run_name=f"roberta-ep{num_epochs_roberta}-lr{learning_rate_roberta}-bs{batch_size_roberta}-samples{n_samples_train}-run{int(time.time())}-no_preprocess",
#     report_to="wandb"
# )

**Initialize Trainers**

Configure Trainer objects for both BERT and RoBERTa using the small training and validation sets, tokenizers, training arguments, and evaluation metric.

In [119]:
# bert_trainer = Trainer(
#     model=bert_model,
#     args=bert_training_args,
#     train_dataset=train_dataset_bert,
#     eval_dataset=val_dataset_bert,
#     tokenizer=bert_tokenizer,
#     compute_metrics=compute_metrics
# )

In [120]:
# roberta_trainer = Trainer(
#     model=roberta_model,
#     args=roberta_training_args,
#     train_dataset=train_dataset_roberta,
#     eval_dataset=val_dataset_roberta,
#     tokenizer=roberta_tokenizer,
#     compute_metrics=compute_metrics
# )

**Model Training**

Train both BERT and RoBERTa models on the small datasets using the Trainer interface.

## Manual Trials

Manually trained BERT and RoBERTa on various sample sizes to understand how accuracy scales:
- 1000, 5000, 10000 train samples for 5 epochs

This section is useful for baseline comparison before running Optuna for automated tuning.


In [121]:
# bert_trainer.train()

In [122]:
# roberta_trainer.train()

# # Safely finish run with specific parameters
# wandb.finish()

**Hyperparameters Tuning with Optuna**

In [123]:
def build_trainer(model_checkpoint, trial, run_prefix, train_dataset, val_dataset):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32])
    num_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    n_samples = len(train_dataset)

    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=5)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    run_name = f"{run_prefix}-ep{num_epochs}-lr{learning_rate}-bs{batch_size}-samples{n_samples}-run{int(time.time())}-w_preprocess"

    args = TrainingArguments(
        output_dir=f"./results/{run_prefix}/{run_name}",
        disable_tqdm=True,
        fp16=True,
        eval_strategy="epoch",
        save_strategy="epoch",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=0.01,
        load_best_model_at_end=True,
        save_total_limit=1,
        logging_strategy="epoch",
        logging_dir=f"./logs/{run_prefix}/{run_name}",
        run_name=run_name,
        report_to="wandb",
        metric_for_best_model="f1_macro",
        greater_is_better=True
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [124]:
def objective_bert(trial):
    trainer = build_trainer(
        model_checkpoint="bert-base-uncased",
        trial=trial,
        run_prefix="bert",
        train_dataset=tokenized_bert_train_sub,
        val_dataset=tokenized_bert_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

def objective_roberta(trial):
    trainer = build_trainer(
        model_checkpoint="roberta-base",
        trial=trial,
        run_prefix="roberta",
        train_dataset=tokenized_roberta_train_sub,
        val_dataset=tokenized_roberta_val_sub
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1_macro"]

In [125]:
study_bert = optuna.create_study(direction="maximize",
                                 pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                 study_name="bert_study_stratify_w_preprocess",
                                 storage="sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/bert_study_stratify_w_preprocess.db",
                                 load_if_exists=True)
study_bert.optimize(objective_bert, n_trials=5)

[I 2025-07-27 21:47:15,620] A new study created in RDB with name: bert_study_stratify_w_preprocess
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5707, 'grad_norm': 7.429507732391357, 'learning_rate': 1.0984125391680366e-05, 'epoch': 1.0}
{'eval_loss': 1.5513710975646973, 'eval_accuracy': 0.294, 'eval_f1_macro': 0.19602703876682515, 'eval_runtime': 0.4366, 'eval_samples_per_second': 1145.131, 'eval_steps_per_second': 36.644, 'epoch': 1.0}
{'loss': 1.4413, 'grad_norm': 6.583423614501953, 'learning_rate': 7.399831842816247e-06, 'epoch': 2.0}
{'eval_loss': 1.4327157735824585, 'eval_accuracy': 0.384, 'eval_f1_macro': 0.3436538882590473, 'eval_runtime': 0.4172, 'eval_samples_per_second': 1198.363, 'eval_steps_per_second': 38.348, 'epoch': 2.0}
{'loss': 1.3092, 'grad_norm': 11.536330223083496, 'learning_rate': 3.757727107680126e-06, 'epoch': 3.0}
{'eval_loss': 1.3724521398544312, 'eval_accuracy': 0.422, 'eval_f1_macro': 0.4098431988317487, 'eval_runtime': 0.4241, 'eval_samples_per_second': 1178.983, 'eval_steps_per_second': 37.727, 'epoch': 3.0}
{'loss': 1.216, 'grad_norm': 12.700071334838867, 'learning_rate': 1.15622372544

[I 2025-07-27 21:47:48,950] Trial 0 finished with value: 0.42585554954587834 and parameters: {'learning_rate': 1.4568418940544487e-05, 'batch_size': 32, 'num_train_epochs': 4}. Best is trial 0 with value: 0.42585554954587834.


{'eval_loss': 1.35353422164917, 'eval_accuracy': 0.434, 'eval_f1_macro': 0.42585554954587834, 'eval_runtime': 0.4423, 'eval_samples_per_second': 1130.553, 'eval_steps_per_second': 36.178, 'epoch': 4.0}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

{'loss': 1.5038, 'grad_norm': 10.170616149902344, 'learning_rate': 1.183070087692655e-05, 'epoch': 1.0}
{'eval_loss': 1.3729413747787476, 'eval_accuracy': 0.378, 'eval_f1_macro': 0.3814512219979995, 'eval_runtime': 1.3644, 'eval_samples_per_second': 366.458, 'eval_steps_per_second': 46.174, 'epoch': 1.0}
{'loss': 1.1726, 'grad_norm': 12.486678123474121, 'learning_rate': 8.88187214389401e-06, 'epoch': 2.0}
{'eval_loss': 1.2073100805282593, 'eval_accuracy': 0.52, 'eval_f1_macro': 0.5304234805579976, 'eval_runtime': 1.3827, 'eval_samples_per_second': 361.622, 'eval_steps_per_second': 45.564, 'epoch': 2.0}
{'loss': 0.898, 'grad_norm': 21.29502296447754, 'learning_rate': 5.93304341086147e-06, 'epoch': 3.0}
{'eval_loss': 1.2406476736068726, 'eval_accuracy': 0.498, 'eval_f1_macro': 0.49909510693501186, 'eval_runtime': 1.4576, 'eval_samples_per_second': 343.027, 'eval_steps_per_second': 43.221, 'epoch': 3.0}
{'loss': 0.6738, 'grad_norm': 9.70681095123291, 'learning_rate': 2.9960099927610604e-0

[I 2025-07-27 21:52:31,760] Trial 3 finished with value: 0.5304234805579976 and parameters: {'learning_rate': 1.47441436651627e-05, 'batch_size': 8, 'num_train_epochs': 5}. Best is trial 3 with value: 0.5304234805579976.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.4401, 'grad_norm': 8.896726608276367, 'learning_rate': 2.1786201916404838e-05, 'epoch': 1.0}
{'eval_loss': 1.3133612871170044, 'eval_accuracy': 0.398, 'eval_f1_macro': 0.38155321206153536, 'eval_runtime': 1.3653, 'eval_samples_per_second': 366.215, 'eval_steps_per_second': 46.143, 'epoch': 1.0}
{'loss': 1.0101, 'grad_norm': 15.091058731079102, 'learning_rate': 1.0979554140410375e-05, 'epoch': 2.0}
{'eval_loss': 1.1193854808807373, 'eval_accuracy': 0.542, 'eval_f1_macro': 0.5578473719711086, 'eval_runtime': 1.3798, 'eval_samples_per_second': 362.375, 'eval_steps_per_second': 45.659, 'epoch': 2.0}
{'loss': 0.6369, 'grad_norm': 18.44041633605957, 'learning_rate': 1.729063644159114e-07, 'epoch': 3.0}
{'eval_loss': 1.1420174837112427, 'eval_accuracy': 0.56, 'eval_f1_macro': 0.5731334451845292, 'eval_runtime': 1.3702, 'eval_samples_per_second': 364.923, 'eval_steps_per_second': 45.98, 'epoch': 3.0}
{'train_runtime': 66.2995, 'train_samples_per_second': 90.498, 'train_steps_per_sec

[I 2025-07-27 21:53:40,631] Trial 4 finished with value: 0.5731334451845292 and parameters: {'learning_rate': 3.241994332798339e-05, 'batch_size': 8, 'num_train_epochs': 3}. Best is trial 4 with value: 0.5731334451845292.


{'eval_loss': 1.1420174837112427, 'eval_accuracy': 0.56, 'eval_f1_macro': 0.5731334451845292, 'eval_runtime': 1.3465, 'eval_samples_per_second': 371.33, 'eval_steps_per_second': 46.788, 'epoch': 3.0}


In [126]:
study_roberta = optuna.create_study(direction="maximize",
                                    pruner=MedianPruner(n_startup_trials=2, n_warmup_steps=1),
                                    study_name="roberta_study_stratify_w_preprocess",
                                    storage="sqlite:////content/drive/MyDrive/Colab Notebooks/nlp_project/optuna/roberta_study_stratify_w_preprocess.db",
                                    load_if_exists=True)
study_roberta.optimize(objective_roberta, n_trials=5)

[I 2025-07-27 21:53:42,171] A new study created in RDB with name: roberta_study_stratify_w_preprocess
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5047, 'grad_norm': 17.84319496154785, 'learning_rate': 7.986974451273315e-06, 'epoch': 1.0}
{'eval_loss': 1.400625467300415, 'eval_accuracy': 0.342, 'eval_f1_macro': 0.34047397985967753, 'eval_runtime': 1.3971, 'eval_samples_per_second': 357.888, 'eval_steps_per_second': 45.094, 'epoch': 1.0}
{'loss': 1.2045, 'grad_norm': 18.301443099975586, 'learning_rate': 5.3422809243616216e-06, 'epoch': 2.0}
{'eval_loss': 1.292461633682251, 'eval_accuracy': 0.432, 'eval_f1_macro': 0.45012255049140293, 'eval_runtime': 1.3795, 'eval_samples_per_second': 362.456, 'eval_steps_per_second': 45.67, 'epoch': 2.0}
{'loss': 0.9985, 'grad_norm': 23.73128318786621, 'learning_rate': 2.6975873974499275e-06, 'epoch': 3.0}
{'eval_loss': 1.3321565389633179, 'eval_accuracy': 0.44, 'eval_f1_macro': 0.45703453941336136, 'eval_runtime': 1.4593, 'eval_samples_per_second': 342.634, 'eval_steps_per_second': 43.172, 'epoch': 3.0}
{'loss': 0.8694, 'grad_norm': 21.534589767456055, 'learning_rate': 5.28938705382338

[I 2025-07-27 21:55:18,662] Trial 0 finished with value: 0.4828958225796941 and parameters: {'learning_rate': 1.0578774107646774e-05, 'batch_size': 8, 'num_train_epochs': 4}. Best is trial 0 with value: 0.4828958225796941.


{'eval_loss': 1.3310420513153076, 'eval_accuracy': 0.466, 'eval_f1_macro': 0.4828958225796941, 'eval_runtime': 1.4026, 'eval_samples_per_second': 356.485, 'eval_steps_per_second': 44.917, 'epoch': 4.0}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

[I 2025-07-27 21:58:58,100] Trial 3 finished with value: 0.5172949202532156 and parameters: {'learning_rate': 3.00934757169666e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 3 with value: 0.5172949202532156.


{'eval_loss': 1.3494945764541626, 'eval_accuracy': 0.498, 'eval_f1_macro': 0.5172949202532156, 'eval_runtime': 0.4292, 'eval_samples_per_second': 1165.027, 'eval_steps_per_second': 37.281, 'epoch': 5.0}


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 1.5323, 'grad_norm': inf, 'learning_rate': 2.2048304869918244e-05, 'epoch': 1.0}
{'eval_loss': 1.393764615058899, 'eval_accuracy': 0.386, 'eval_f1_macro': 0.3572182187430565, 'eval_runtime': 0.4374, 'eval_samples_per_second': 1143.178, 'eval_steps_per_second': 36.582, 'epoch': 1.0}
{'loss': 1.2366, 'grad_norm': 15.865453720092773, 'learning_rate': 1.6666435177261037e-05, 'epoch': 2.0}
{'eval_loss': 1.2744221687316895, 'eval_accuracy': 0.448, 'eval_f1_macro': 0.4659571519267809, 'eval_runtime': 0.4231, 'eval_samples_per_second': 1181.616, 'eval_steps_per_second': 37.812, 'epoch': 2.0}
{'loss': 1.0, 'grad_norm': 24.58997917175293, 'learning_rate': 1.1197761134722257e-05, 'epoch': 3.0}
{'eval_loss': 1.258523941040039, 'eval_accuracy': 0.478, 'eval_f1_macro': 0.49358036006984635, 'eval_runtime': 0.4238, 'eval_samples_per_second': 1179.781, 'eval_steps_per_second': 37.753, 'epoch': 3.0}
{'loss': 0.816, 'grad_norm': 25.1953125, 'learning_rate': 5.729087092183481e-06, 'epoch': 4.0}
{

[I 2025-07-27 21:59:44,849] Trial 4 finished with value: 0.49797889821977953 and parameters: {'learning_rate': 2.7343370212693884e-05, 'batch_size': 32, 'num_train_epochs': 5}. Best is trial 3 with value: 0.5172949202532156.


In [127]:
best_trial_bert = study_bert.best_trial
print('Bert best trial on subset:')
print(best_trial_bert.params)
best_trial_roberta = study_roberta.best_trial
print('RoBerta best trial on subset:')
print(best_trial_roberta.params)

Bert best trial on subset:
{'learning_rate': 3.241994332798339e-05, 'batch_size': 8, 'num_train_epochs': 3}
RoBerta best trial on subset:
{'learning_rate': 3.00934757169666e-05, 'batch_size': 32, 'num_train_epochs': 5}


In [128]:
final_trainer_bert = build_trainer(
    model_checkpoint="bert-base-uncased",
    trial=best_trial_bert,
    run_prefix="bert_final_stratify_w_preprocess",
    train_dataset=tokenized_bert_train,
    val_dataset=tokenized_bert_val
)
final_trainer_bert.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 0.7319, 'grad_norm': 122.37641906738281, 'learning_rate': 2.1627998474133138e-05, 'epoch': 1.0}
{'eval_loss': 0.5761656761169434, 'eval_accuracy': 0.814902580305424, 'eval_f1_macro': 0.8212106741617727, 'eval_runtime': 10.2924, 'eval_samples_per_second': 369.008, 'eval_steps_per_second': 46.15, 'epoch': 1.0}
{'loss': 0.4138, 'grad_norm': 4.548095226287842, 'learning_rate': 1.082555153303702e-05, 'epoch': 2.0}
{'eval_loss': 0.5603269934654236, 'eval_accuracy': 0.8507109004739336, 'eval_f1_macro': 0.8563363631079952, 'eval_runtime': 10.3404, 'eval_samples_per_second': 367.297, 'eval_steps_per_second': 45.936, 'epoch': 2.0}
{'loss': 0.2758, 'grad_norm': 0.13800150156021118, 'learning_rate': 2.310459194090167e-08, 'epoch': 3.0}
{'eval_loss': 0.6360498666763306, 'eval_accuracy': 0.8512374934175883, 'eval_f1_macro': 0.8552087104829782, 'eval_runtime': 10.349, 'eval_samples_per_second': 366.991, 'eval_steps_per_second': 45.898, 'epoch': 3.0}
{'train_runtime': 1227.3237, 'train_sample

TrainOutput(global_step=15435, training_loss=0.4738183498845967, metrics={'train_runtime': 1227.3237, 'train_samples_per_second': 100.602, 'train_steps_per_second': 12.576, 'train_loss': 0.4738183498845967, 'epoch': 3.0})

In [129]:
final_trainer_roberta = build_trainer(
    model_checkpoint="roberta-base",
    trial=best_trial_roberta,
    run_prefix="roberta_final_stratify_w_preprocess",
    train_dataset=tokenized_roberta_train,
    val_dataset=tokenized_roberta_val
)
final_trainer_roberta.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-123-459110852.py:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 0.875, 'grad_norm': 40.04572677612305, 'learning_rate': 2.4088810165982123e-05, 'epoch': 1.0}
{'eval_loss': 0.7696745991706848, 'eval_accuracy': 0.7101105845181674, 'eval_f1_macro': 0.7135517580361457, 'eval_runtime': 3.0536, 'eval_samples_per_second': 1243.785, 'eval_steps_per_second': 38.971, 'epoch': 1.0}
{'loss': 0.529, 'grad_norm': 38.62520217895508, 'learning_rate': 1.807479155339175e-05, 'epoch': 2.0}
{'eval_loss': 0.6237698197364807, 'eval_accuracy': 0.7754081095313323, 'eval_f1_macro': 0.7831979755402771, 'eval_runtime': 3.0402, 'eval_samples_per_second': 1249.244, 'eval_steps_per_second': 39.142, 'epoch': 2.0}
{'loss': 0.3824, 'grad_norm': 44.8629150390625, 'learning_rate': 1.2060772940801378e-05, 'epoch': 3.0}
{'eval_loss': 0.6089505553245544, 'eval_accuracy': 0.791205897840969, 'eval_f1_macro': 0.7987148024675961, 'eval_runtime': 3.1378, 'eval_samples_per_second': 1210.402, 'eval_steps_per_second': 37.925, 'epoch': 3.0}
{'loss': 0.2912, 'grad_norm': 45.473926544189

TrainOutput(global_step=6435, training_loss=0.4597655111815268, metrics={'train_runtime': 610.254, 'train_samples_per_second': 337.212, 'train_steps_per_second': 10.545, 'train_loss': 0.4597655111815268, 'epoch': 5.0})

In [130]:
final_trainer_bert.save_model("models/bert_final_stratify_w_preprocess")
bert_tokenizer.save_pretrained("models/bert_final_stratify_w_preprocess")
!cp -r models/bert_final_stratify_w_preprocess "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/bert_best_model_stratify_w_preprocess"

final_trainer_roberta.save_model("models/roberta_final_stratify_w_preprocess")
roberta_tokenizer.save_pretrained("models/roberta_final_stratify_w_preprocess")
!cp -r models/roberta_final_stratify_w_preprocess "/content/drive/MyDrive/Colab Notebooks/nlp_project/models/roberta_best_model_stratify_w_preprocess"